<a href="https://colab.research.google.com/github/VincentCCL/MTAT/blob/main/MTAT26_COMET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMET Notebook


First we need to install some specific packages.

In [ ]:
!pip -q install "transformers==4.41.2" "huggingface-hub==0.23.4"
!pip -q install unbabel-comet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.23.4 which is incompatible.
diffusers 0.36.0 requires huggingface-hub<2.0,>=0.34.0, but you have huggingface-hub 0.23.4 which is incompatible.
datasets 4.0.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.23.4 which is incompatible.
peft 0.18.1 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s et

##**IMPORTANT: Restart RUNTIME**

And here is the python code for a single sentence. Note that COMET expects a list of sentence-level dictionaries, one dictionary per sentence.
So for multiple sentences, you just have multiple dictionaries in the list.

In [ ]:
import torch
from comet import download_model, load_from_checkpoint

ckpt = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(ckpt)

gpus = 1 if torch.cuda.is_available() else 0

data = [
    {"src": "She eats apples.", "mt": "Zij eet appels.", "ref": "Zij eet appels."}
]

out = model.predict(data, batch_size=8, gpus=gpus)
print("\n",out.system_score)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, u


 0.9804218411445618


## Command line example

We have stored our bing / google / deepl translations of the 30 first sentences of the Tatoeba-en-nl dataset in our google drive.

We download the full source file `dev.en` and the full reference file `dev.nl` (1000 sentences each)

In [ ]:
!wget https://www.ccl.kuleuven.be/~vincent/MTAT/dev.en
!wget https://www.ccl.kuleuven.be/~vincent/MTAT/dev.nl

--2026-02-04 14:38:24--  https://www.ccl.kuleuven.be/~vincent/MTAT/dev.en
Resolving www.ccl.kuleuven.be (www.ccl.kuleuven.be)... 134.58.16.193
Connecting to www.ccl.kuleuven.be (www.ccl.kuleuven.be)|134.58.16.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32441 (32K) [text/plain]
Saving to: ‘dev.en’

dev.en              100%[===================>]  31.68K  --.-KB/s    in 0.1s    

2026-02-04 14:38:25 (218 KB/s) - ‘dev.en’ saved [32441/32441]

--2026-02-04 14:38:25--  https://www.ccl.kuleuven.be/~vincent/MTAT/dev.nl
Resolving www.ccl.kuleuven.be (www.ccl.kuleuven.be)... 134.58.16.193
Connecting to www.ccl.kuleuven.be (www.ccl.kuleuven.be)|134.58.16.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34843 (34K)
Saving to: ‘dev.nl’

dev.nl              100%[===================>]  34.03K  --.-KB/s    in 0.1s    

2026-02-04 14:38:25 (237 KB/s) - ‘dev.nl’ saved [34843/34843]



We use the `head` command to create new files containing only the first 30 lines.

In [ ]:
!head -n 30 dev.en > dev30.en
!head -n 30 dev.nl > dev30.nl

Then we download a python script that calculates the COMET score

In [ ]:
!wget https://www.ccl.kuleuven.be/~vincent/MTAT/comet_score_files.py

--2026-02-04 14:48:07--  https://www.ccl.kuleuven.be/~vincent/MTAT/comet_score_files.py
Resolving www.ccl.kuleuven.be (www.ccl.kuleuven.be)... 134.58.16.193
Connecting to www.ccl.kuleuven.be (www.ccl.kuleuven.be)|134.58.16.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2015 (2.0K)
Saving to: ‘comet_score_files.py.1’

comet_score_files.p 100%[===================>]   1.97K  --.-KB/s    in 0s      

2026-02-04 14:48:08 (482 MB/s) - ‘comet_score_files.py.1’ saved [2015/2015]



And then we run the script with three arguments:
1. the source file `dev30.en`
2. the mt output, which is stored on our google drive
3. the reference file `dev30.nl`

In [ ]:
!python comet_score_files.py dev30.en \
/content/drive/MyDrive/Vakken/MTAT/2026/tatoeba/tatoeba-bing.txt \
dev30.nl

Fetching 5 files: 100% 5/5 [00:00<00:00, 10330.80it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Encoder model frozen.
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and ex